In [1]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# ----------------------------
# CONNECT TO VLLM ENDPOINT
# ----------------------------
client = OpenAI(
    base_url="https://gpt-oss-120b.vailsys.net/v1",
    api_key="EMPTY"
)

# ----------------------------
# SYSTEM PROMPT (PLAIN STYLE)
# ----------------------------
system_prompt = """
You are a careful English writer.

Your task is to produce ONE clear, natural-sounding English sentence.

Requirements:
1. The exact substring provided by the user MUST appear verbatim.
2. Treat the substring strictly as a NOUN PHRASE.
3. Do NOT interpret any word inside the substring as a verb or action.
4. You may add words before or after the substring, but NEVER inside it.
5. Write a normal, everyday sentence that could appear in news, academic prose,
   or casual written English.
6. Avoid poetic language, humor, metaphors, or unusual imagery.
7. Output ONLY the sentence. Do not add explanations or commentary.
"""

# ----------------------------
# GENERATION FUNCTION
# ----------------------------
def generate_sentence(model, system_prompt, phrase):

    user_prompt = (
        f"Write one simple English sentence that includes exactly this substring: {phrase}."
    )

    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            max_tokens=5000,
            temperature=0.3,
            top_p=0.9,
            n=1,
        )
    except Exception as e:
        print("⚠ ERROR:", e)
        return f"This sentence contains the required phrase: {phrase}"

    if not resp or not resp.choices:
        return f"This sentence contains the required phrase: {phrase}"

    content = resp.choices[0].message.content.strip()

    if phrase not in content:
        return f"This sentence contains the required phrase: {phrase}"

    return content


# # ----------------------------
# # PIPELINE ON CSV / DATAFRAME
# # ----------------------------
# input_csv = "hand_coded_corpus_finetuning_with_model_preds.csv"
# output_csv = "hand_coded_output_with_generated_sentences.csv"

# model = "/models/gpt-oss-120b"

# df = pd.read_csv(input_csv)

# sentences = []
# for phrase in tqdm(df["phrase"]):
#     sentence = generate_sentence(model, system_prompt, phrase)
#     sentences.append(sentence)

# df["generated_sentence"] = sentences
# df.to_csv(output_csv, index=False)


In [2]:

# df = pd.read_csv(input_csv)
# assert "predicted_resp" in df.columns
# #df = df.head(100)
# # ----------------------------
# # MULTITHREADED PROCESSING
# # ----------------------------
# def process_row(i):
#     phrase = str(df.at[i, "predicted_resp"])
#     return generate_sentence(model, system_prompt, phrase)

# indices = list(range(len(df)))

# with ThreadPoolExecutor(max_workers=8) as ex:
#     sentences = list(tqdm(ex.map(process_row, indices), total=len(df)))

# df["GeneratedSentence"] = sentences

# # Save output
# df.to_csv(output_csv, index=False)
# print(f"\n✨ Done! Saved annotated CSV to: {output_csv}")

In [3]:
import pandas as pd
import os

input_csv = "all_binoms_with_model_preds.csv"
chunk_size = 10000

df = pd.read_csv(input_csv)
os.makedirs("chunks_all_binoms", exist_ok=True)

for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i+chunk_size]
    chunk.to_csv(f"chunks_all_binoms/chunk_{i//chunk_size:04d}.csv", index=False)

print("Done splitting!")


Done splitting!


In [ ]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI
import os

client = OpenAI(base_url="https://gpt-oss-120b.vailsys.net/v1", api_key="EMPTY")

# your generate_sentence() and all prompts stay the same

def process_chunk(chunk_path):
    print(f"🔹 Processing {chunk_path}")

    df = pd.read_csv(chunk_path)
    model = "/models/gpt-oss-120b"

    def process_row(i):
        phrase = str(df.at[i, "predicted_resp"])
        #print(phrase)
        return generate_sentence(model, system_prompt, phrase)

    indices = range(len(df))

    with ThreadPoolExecutor(max_workers=8) as ex:
        sentences = list(tqdm(ex.map(process_row, indices), total=len(df)))

    df["GeneratedSentence"] = sentences

    # output file path
    out_path = chunk_path.replace("chunks_all_binoms/", "processed_all_binoms/")
    os.makedirs("processed_all_binoms", exist_ok=True)
    df.to_csv(out_path, index=False)

    print(f"✔ Finished {chunk_path} → {out_path}")


import os

chunk_files = sorted(os.listdir("chunks_all_binoms"))
os.makedirs("processed_all_binoms", exist_ok=True)

for f in chunk_files:
    chunk_path = f"chunks_all_binoms/{f}"
    out_path = f"processed_all_binoms/{f}"

    # ⛔ Skip if already processed
    if os.path.exists(out_path):
        print(f"⏩ Already done: {out_path}")
        continue

    try:
        process_chunk(chunk_path)
    except Exception as e:
        print(f"\n💥 Failed on {chunk_path}: {e}")
        print("Stopping so you can resume later.")
        break


🔹 Processing chunks_all_binoms/chunk_0000.csv


  2%|█▉                                                                             | 242/10000 [00:47<24:38,  6.60it/s]

In [12]:
import pandas as pd
import glob

files = sorted(glob.glob("processed_all_binoms/chunk_*.csv"))
dfs = [pd.read_csv(f) for f in files]
merged = pd.concat(dfs, ignore_index=True)

merged.to_csv("finetuning_condition_all_binoms.csv", index=False)
print("✨ Merged final file saved!")


✨ Merged final file saved!


In [10]:
1+1

2

In [25]:
import pandas as pd
import os

input_csv = "fintetuning-corpus-with-model-preds-relfreq.csv"
chunk_size = 1000

df = pd.read_csv(input_csv)
os.makedirs("chunks_relfreq", exist_ok=True)

for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i+chunk_size]
    chunk.to_csv(f"chunks_relfreq/chunk_{i//chunk_size:04d}.csv", index=False)

print("Done splitting!")


import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI
import os

client = OpenAI(base_url="https://gpt-oss-120b.vailsys.net/v1", api_key="EMPTY")

# your generate_sentence() and all prompts stay the same

def process_chunk(chunk_path):
    print(f"🔹 Processing {chunk_path}")

    df = pd.read_csv(chunk_path)
    model = "/models/gpt-oss-120b"

    def process_row(i):
        phrase = str(df.at[i, "relfreq_resp"])
        return generate_sentence(model, system_prompt, phrase)

    indices = range(len(df))

    with ThreadPoolExecutor(max_workers=8) as ex:
        sentences = list(tqdm(ex.map(process_row, indices), total=len(df)))

    df["GeneratedSentence"] = sentences

    # output file path
    out_path = chunk_path.replace("chunks_relfreq/", "processed_relfreq/")
    os.makedirs("processed_relfreq", exist_ok=True)
    df.to_csv(out_path, index=False)

    print(f"✔ Finished {chunk_path} → {out_path}")


import os

chunk_files = sorted(os.listdir("chunks_relfreq"))
os.makedirs("processed_relfreq", exist_ok=True)

for f in chunk_files:
    chunk_path = f"chunks_relfreq/{f}"
    out_path = chunk_path.replace("chunks_relfreq/", "processed_relfreq/")

    # ⛔ Skip if already processed
    if os.path.exists(out_path):
        print(f"⏩ Already done: {out_path}")
        continue

    try:
        process_chunk(chunk_path)
    except Exception as e:
        print(f"\n💥 Failed on {chunk_path}: {e}")
        print("Stopping so you can resume later.")
        break



Done splitting!
⏩ Already done: processed_relfreq/chunk_0000.csv
⏩ Already done: processed_relfreq/chunk_0001.csv
⏩ Already done: processed_relfreq/chunk_0002.csv
⏩ Already done: processed_relfreq/chunk_0003.csv
⏩ Already done: processed_relfreq/chunk_0004.csv
⏩ Already done: processed_relfreq/chunk_0005.csv
⏩ Already done: processed_relfreq/chunk_0006.csv
⏩ Already done: processed_relfreq/chunk_0007.csv
⏩ Already done: processed_relfreq/chunk_0008.csv
⏩ Already done: processed_relfreq/chunk_0009.csv
⏩ Already done: processed_relfreq/chunk_0010.csv
⏩ Already done: processed_relfreq/chunk_0011.csv
⏩ Already done: processed_relfreq/chunk_0012.csv
⏩ Already done: processed_relfreq/chunk_0013.csv
⏩ Already done: processed_relfreq/chunk_0014.csv
⏩ Already done: processed_relfreq/chunk_0015.csv
⏩ Already done: processed_relfreq/chunk_0016.csv
⏩ Already done: processed_relfreq/chunk_0017.csv
⏩ Already done: processed_relfreq/chunk_0018.csv
⏩ Already done: processed_relfreq/chunk_0019.csv
⏩ Al


00%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:09<00:00,  1.82it/s]

✔ Finished chunks_relfreq/chunk_0044.csv → processed_relfreq/chunk_0044.csv
🔹 Processing chunks_relfreq/chunk_0045.csv



00%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:03<00:00,  1.84it/s]

✔ Finished chunks_relfreq/chunk_0045.csv → processed_relfreq/chunk_0045.csv
🔹 Processing chunks_relfreq/chunk_0046.csv


...

✔ Finished chunks_relfreq/chunk_0047.csv → processed_relfreq/chunk_0047.csv
🔹 Processing chunks_relfreq/chunk_0048.csv



00%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:07<00:00,  1.83it/s]

✔ Finished chunks_relfreq/chunk_0048.csv → processed_relfreq/chunk_0048.csv
🔹 Processing chunks_relfreq/chunk_0049.csv


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 710/710 [06:12<00:00,  1.91it/s]

✔ Finished chunks_relfreq/chunk_0049.csv → processed_relfreq/chunk_0049.csv


In [26]:
import pandas as pd
import glob

files = sorted(glob.glob("processed_relfreq/chunk_*.csv"))
dfs = [pd.read_csv(f) for f in files]
merged = pd.concat(dfs, ignore_index=True)

merged.to_csv("finetuning_condition_relfreq.csv", index=False)
print("✨ Merged final file saved!")


✨ Merged final file saved!
